# Pathway
> [Pathway](https://pathway.com/) 是一个开源数据处理框架。它使您能够轻松开发数据转换管道和机器学习应用程序，这些应用程序可以处理实时数据源和不断变化的数据。

本 Notebook 演示了如何将实时 `Pathway` 数据索引管道与 `Langchain` 结合使用。您可以像查询常规向量存储一样，从您的链中查询此管道的结果。然而，在底层，Pathway 会在每次数据更改时更新索引，从而始终为您提供最新的答案。

在本 Notebook 中，我们将使用一个 [公共演示文档处理管道](https://pathway.com/solutions/ai-pipelines#try-it-out)，该管道：

1. 监控多个云数据源的数据变化。
2. 为数据构建向量索引。

要拥有自己的文档处理管道，请查看 [托管服务](https://pathway.com/solutions/ai-pipelines) 或 [自行构建](https://pathway.com/developers/user-guide/llm-xpack/vectorstore_pipeline/)。

我们将使用 `VectorStore` 客户端连接到索引，该客户端实现了 `similarity_search` 函数来检索匹配的文档。

本文档使用的基本管道允许轻松地构建存储在云位置的文件的简单向量索引。但是，Pathway 提供了构建实时数据管道和应用程序所需的一切，包括类似 SQL 的操作，如 group by 归约和不同数据源之间的联接、基于时间的 grouping 和数据窗口化，以及各种连接器。

您需要安装 `langchain-community` (`pip install -qU langchain-community`) 才能使用此集成。

## 查询数据管道

要实例化和配置客户端，您需要提供文档索引管道的 `url` 或 `host` 和 `port`。在下面的代码中，我们使用了一个公开可用的 [演示管道](https://pathway.com/solutions/ai-pipelines#try-it-out)，其 REST API 可在 `https://demo-document-indexing.pathway.stream` 访问。此演示从 [Google Drive](https://drive.google.com/drive/u/0/folders/1cULDv2OaViJBmOfG5WB0oWcgayNrGtVs) 和 [Sharepoint](https://navalgo.sharepoint.com/sites/ConnectorSandbox/Shared%20Documents/Forms/AllItems.aspx?id=%2Fsites%2FConnectorSandbox%2FShared%20Documents%2FIndexerSandbox&p=true&ga=1) 摄取文档，并维护一个用于检索文档的索引。

In [ ]:
from langchain_community.vectorstores import PathwayVectorClient

client = PathwayVectorClient(url="https://demo-document-indexing.pathway.stream")

然后我们就可以开始提问了

In [ ]:
query = "What is Pathway?"
docs = client.similarity_search(query)

In [ ]:
print(docs[0].page_content)

**轮到你了！** [获取你的 pipeline](https://pathway.com/solutions/ai-pipelines) 或将 [新文档](https://chat-realtime-sharepoint-gdrive.demo.pathway.com/) 上传到演示 pipeline 中并重试查询！

## 基于文件元数据进行筛选

我们支持使用 [jmespath](https://jmespath.org/) 表达式对文档进行筛选，例如：

In [ ]:
# take into account only sources modified later than unix timestamp
docs = client.similarity_search(query, metadata_filter="modified_at >= `1702672093`")

# take into account only sources modified later than unix timestamp
docs = client.similarity_search(query, metadata_filter="owner == `james`")

# take into account only sources with path containing 'repo_readme'
docs = client.similarity_search(query, metadata_filter="contains(path, 'repo_readme')")

# and of two conditions
docs = client.similarity_search(
    query, metadata_filter="owner == `james` && modified_at >= `1702672093`"
)

# or of two conditions
docs = client.similarity_search(
    query, metadata_filter="owner == `james` || modified_at >= `1702672093`"
)

## 获取已索引文件的信息

`PathwayVectorClient.get_vectorstore_statistics()` 提供了关于向量存储状态的关键统计信息，例如索引文件的数量和最后一个更新时间戳。你可以在你的链中使用它来告知用户你的知识库有多新。

In [ ]:
client.get_vectorstore_statistics()

## 你自己的处理流程

### 在生产环境中运行

要拥有自己的 Pathway 数据索引管道，请查看 Pathway 的[托管管道优惠](https://pathway.com/solutions/ai-pipelines)。您也可以运行自己的 Pathway 管道——有关如何构建管道的信息，请参阅[Pathway 指南](https://pathway.com/developers/user-guide/llm-xpack/vectorstore_pipeline/)。

### 处理文档

向量化管道支持用于解析、拆分和嵌入文档的可插拔组件。对于嵌入和拆分，您可以使用 [Langchain 组件](https://pathway.com/developers/user-guide/llm-xpack/vectorstore_pipeline/#langchain)，或者查看 Pathway 中提供的 [embedders](https://pathway.com/developers/api-docs/pathway-xpacks-llm/embedders) 和 [splitters](https://pathway.com/developers/api-docs/pathway-xpacks-llm/splitters)。如果未提供解析器，则默认为 `UTF-8` 解析器。您可以在 [此处](https://github.com/pathwaycom/pathway/blob/main/python/pathway/xpacks/llm/parser.py) 找到可用的解析器。